## Activité 3 : Analyse d’un complexe avec PyMOL

On ajoute un élément au `python search path` afin de pouvoir utiliser nos 
fonctions et modules :

In [1]:
import sys

TXT_PROJ_ROOT = "/home/gml/Documents/Master/M1_BIBS/S2/Structurale/"

if TXT_PROJ_ROOT not in sys.path:
    sys.path.append(TXT_PROJ_ROOT)
    print("Added project root to Python search path")
else:
    print("Project root already present in Python search path")

Added project root to Python search path


Classes pour manipuler plus facilement les chemins d'accès en Python :

In [2]:
from piestruct.utils.customobjs import (ObjDict as odict, Path as path)

Modules nécessaires pour faire l'analyse structurale :

In [3]:
import Bio.PDB as pdb
import freesasa
import nglview as nv

In [4]:
proj_root = path(TXT_PROJ_ROOT)

In [5]:
pdb_files = proj_root.joinpath("data").dglob("*.pdb")
pdb_files

{'1he8.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/1he8.pdb'),
 '1HE8_AB_c.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/1HE8_AB_c.pdb'),
 '1e8y.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/1e8y.pdb'),
 '121p.pdb': PosixPath('/home/gml/Documents/Master/M1_BIBS/S2/Structurale/data/121p.pdb')}

## Créer les structures

In [21]:
(prot_s_sasa := freesasa.Structure( pdb_files["1HE8_AB_c.pdb"].abs )) 

In [22]:
(prot_c_sasa := freesasa.calc(prot_s_sasa))

In [23]:
# Attention : J'ai mis l'argument QUIET=True pour cacher les alertes issues de l'importation
# du fichier. D'après la doc officielle de BioPython, ces "Warnings" sont des indicateurs
# de problèmes dans la structure du fichier PDB. À RETENIR.
(prot_biopy := pdb.PDBParser(QUIET=True).get_structure(id="1HE8", file=pdb_files["1HE8_AB_c.pdb"].abs))

<Structure id=1HE8>

In [18]:
prot_biopy.get_chains()

[<Chain id=A>, <Chain id=B>]

In [50]:
(view_A := nv.show_biopython(prot_biopy[0]["A"]))

NGLWidget()

In [48]:
(view_B := nv.show_biopython(prot_biopy[0]["B"]))

NGLWidget()

In [49]:
(view_full := nv.show_biopython(prot_biopy))

NGLWidget()

### Tâche 3.1

1. De quelles protéines s’agit-il ?

Il s'agit du complexe Ras G12V - PI 3-kinase gamma. C'est un hétéro-2-mère A1B1 asymétrique, dont la fonction est importante pour la survie de cellules transformées (_Ras activation of phosphoinositide 3-kinase (PI3K) is important for survival of transformed cells_).

2. Combien de chaînes polypeptidiques contient le fichier PDB ? Combien de résidus par chaîne ?

In [25]:
sum(1 for chain in prot_biopy.get_chains())

2

In [36]:
prot_resid_areas = prot_c_sasa.residueAreas()

In [38]:
for chain in prot_resid_areas.keys():
    print(f"chain {chain} : {sum(1 for residue in prot_resid_areas[chain])} residues")

chain A : 749 residues
chain B : 166 residues


La structure PDB `1HE8_AB_c` contient deux chaînes, A et B. La chaîne A contient 766 résidus, c'est-à-dire plus de quatre fois plus que la chaîne B. Ceci nous mène à dire qu'il s'agit d'un hétéro-dimère. Ceci est en accord avec les informations trouvées sur la fiche `RCSB PDB`.

### Tâche 3.2
Allez voir les fiches Uniprot correspondantes aux deux monomères en utilisant les liens dans la fiche du
PDB

Pour chaque protéine dans le complexe, répondez aux questions suivantes :
1. La structure PDB représente-t-elle la protéine entière ou une portion ?
2. La protéine est composée de plusieurs domaines ?
3. Quelle est la fonction (putative) de la protéine ? et de ses domaines ?